In [ ]:
import json
import geopandas as gp
import pandas as pd
import numpy as np
from data.fetch_data import get_dataframe
from bokeh.io import output_notebook, show, output_file, export_png, push_notebook
from bokeh.plotting import figure, gmap
from bokeh.models import GeoJSONDataSource, GMapOptions, ColumnDataSource, WheelZoomTool, LinearColorMapper, ColorBar, CustomJS
from bokeh.palettes import mpl, viridis
from bokeh.tile_providers import get_provider, Vendors
from bokeh.transform import factor_cmap
from bokeh.models.widgets import Slider, Toggle
from bokeh.layouts import column, grid, Spacer
from bokeh.models.tools import HoverTool

In [ ]:
# # Filter and drop null rows
# filt = ["date", "block", "primary_type", "description", "location_description", "arrest", "year", "latitude", "longitude"]
# df = get_dataframe("chicago_crime", "crime")[filt].dropna()

# # Extract time info into seperate columns for easy querying
# df["year"] = df["year"].astype("int16")
# df["month"] = df.date.apply(lambda x: x.month).astype("int8")
# df["day"] = df.date.apply(lambda x: x.day).astype("int8")
# df["hour"] = df.date.apply(lambda x: x.hour).astype("int8")

# # Create geometry from lat and long, project to the web mercator, and drop unnecessary rows
# gdf = gp.GeoDataFrame(df, geometry=gp.points_from_xy(df.longitude, df.latitude), crs={'init': 'epsg:4326'})
# gdf = gdf.to_crs({'init': 'epsg:3857'})
# gdf = gdf.drop(["latitude", "longitude", "date"], axis=1)
gdf = gp.read_file("data/chicago_crime.geojson")

In [ ]:
# gdf.to_file("chicago_crime.geojson", driver='GeoJSON')

In [ ]:
gdf2018 = gdf[gdf.year == 2018]
gdfSMALL = gdf2018[["primary_type", "description", "month", "geometry"]]
months = {}
counts = {}
for i in range(1,13):
    group = gdf2018[gdf2018.month == i].groupby("primary_type").count().sort_values("year",ascending=False).reset_index()[:10]
    counts[i] = {}
    counts[i]['x'] = list(group["primary_type"])
    counts[i]['y'] = list(group["year"])
    months[i] = gdfSMALL[gdfSMALL.month == i].to_json()

In [ ]:
bound = gp.read_file("./data/comm_areas.geojson").rename({"community": "name"}, axis=1).to_crs({'init': 'epsg:3857'})
bound = bound[["name", "area_num_1", "geometry"]].rename({"area_num_1": "com_num"}, axis=1)
bound["com_num"] = bound["com_num"].astype("int8")

pops = pd.read_csv("data/Census-Data-by-Chicago-Community-Area-2018.csv")[["Community", "Community Number", "Total Population"]].rename({"Total Population": "pop"}, axis=1)
# pops = pops.rename({"Community Number": "comnum"}, axis=1)
pops["Community Number"] = pops["Community Number"].astype("int8")
pops["pop"] = pops["pop"].apply(lambda x: x.replace(',' , '')).astype("int64")

bound = bound.merge(pops, left_on="com_num",right_on="Community Number")[["Community", "com_num", "pop", "geometry"]]

In [ ]:
# crime_nums = {}
# for y in range(2016,2019):
#     crime_nums[y] = {}
#     for m in range(1,13):
#         print(y, m, end=' ')
#         crime_nums[y][m] = {}
#         filt = (gdf["year"] == y) & (gdf["month"] == m)
#         geom = gdf[filt].geometry
#         for _, row in bound.iterrows():
#             cn = row["com_num"]
#             com = row["geometry"]
#             crime_nums[y][m][cn] = 0
#             for point in geom:
#                 if com.contains(point):
#                     crime_nums[y][m][cn] += 1
#         print("DONE")
with open('data/crime_nums.json') as fp:
     crime_nums = json.load(fp)

In [ ]:
counts[1]['x']

In [ ]:
# Init figure with map tile
tools = "pan,wheel_zoom,box_zoom,reset"
p = figure(x_axis_type="mercator", y_axis_type="mercator",  plot_height=900, plot_width=700, title="Chicago Crime 2018", tools=tools, 
           active_scroll="wheel_zoom")
p.add_tile(get_provider(Vendors.CARTODBPOSITRON_RETINA))

# Add hover tool
hover = HoverTool(names=["crime_points"], tooltips=[("type", "@primary_type"),("desc", "@description")])
p.add_tools(hover)

# Add crime data points
start_month = 6
crime_s = GeoJSONDataSource(geojson = months[start_month])
p.circle(x='x', y='y', radius=8, color='red', source = crime_s, name="crime_points")

# Make bar graph
p2 = figure(x_range=group.primary_type, toolbar_location=None, title="Crime Counts", plot_height=900, plot_width=900)
p2.xgrid.grid_line_color = None

bar_s = ColumnDataSource(data=counts[start_month])
p2.vbar(x='x', top='y', width=0.9, source=bar_s,
       line_color='white', fill_color=factor_cmap('x', palette=viridis(len(group)), factors=counts[1]['x'])) # , legend_field="primary_type"

# Create slider and callback
slider = Slider(start=1, end=12, value=start_month, step=1, title='Month')
s_callback = CustomJS(args=dict(crime_s=crime_s, months=months, bar_s=bar_s, counts=counts), code="""
    var N = cb_obj.value
    crime_s.geojson = months[N]
    crime_s.change.emit();
    bar_s.data = counts[N]
    bar_s.change.emit();
""")
slider.js_on_change('value', s_callback)

# Add community area boundaries
bound_s = GeoJSONDataSource(geojson = bound.to_json())
comm_areas = p.patches('xs', 'ys', source = bound_s, line_color = 'blue', line_width = 0.5, fill_alpha=0)

toggle = Toggle(label="Community Areas", button_type="success", active=True)
t_callback = CustomJS(args=dict(patch=comm_areas), code="""
    var state = cb_obj.active
    patch.visible = state
    patch.change.emit();
""")
toggle.js_on_click(t_callback)

# Arrange plots and widgets in layouts
layout = grid(children=[[p, p2], [slider, Spacer()], [toggle, Spacer()]])
# output_file("test3.html")
output_notebook()
show(layout)

In [ ]:
bar_s.column_names

In [ ]:
gdf2018.head()

In [ ]:
bound.head()

In [ ]:
print(plot.x_range.to_json(include_defaults=True)

In [ ]:
group = df2018[df2018.month==6].groupby("primary_type").count().sort_values("year",ascending=False).reset_index()[:10]
source = ColumnDataSource(data=group)

p2 = figure(x_range=group.primary_type, toolbar_location=None, title="Crime Counts", plot_height=900)
x = p2.vbar(x='primary_type', top='year', width=0.9, source=source,
       line_color='white', fill_color=factor_cmap('primary_type', palette=viridis(len(group)), factors=group.primary_type)) # , legend_field="primary_type"

p2.xgrid.grid_line_color = None
# p2.y_range.start = 0
# p2.y_range.end = 9
# p2.legend.orientation = "horizontal"
# p2.legend.location = "top_center"
# ColumnDataSource(data=group[:new])

show(p2)
